In [1]:
# Import packages needed to gather, clean, 
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import glob
from config import *

## Gathering Data
* Import the weratedogs csv file into a dataframe

* Download udacity image predictor file from the the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

* Create a json file containing each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

In [2]:
df_ta = pd.read_csv('twitter-archive-enhanced.csv')
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [3]:
# Create a request to pull the image predictions tsv file from the udacity site. Take the image predictions file thats stored in
# check response status to ensure we don't have an error.
# the response output and open a tsv file to hold the data so it can be called later.
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
r.raise_for_status()

with open('image_predictions.tsv', 'wb') as handle:
    for block in r.iter_content(1024):
        handle.write(block)

In [4]:
# Create dataframe for the image predictions
df_pred = pd.read_csv('image_predictions.tsv', sep='\t')

In [6]:
# Pull information on tweets using twitter api
# Pause loop to allow refresh of data pulling
# print twitter tweet json data to file
import time
start = time.time()
print(start)
auth = tweepy.OAuthHandler(consumer_token, consumer_token_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
i = 0
df_error = []
with open('tweet_json.txt', 'w') as append:
    for index, row in df_ta.iterrows():
            try:
                tweet = api.get_status(row['tweet_id'], )
                json.dump(tweet._json, append)
                append.write('\n')
            except tweepy.TweepError as e:
                df_error.append({'tweet_id': row['tweet_id'], 'Error': (e.args[0][0]['code'], e.args[0][0]['message'])})
                i = i+1
                pass
end = time.time() - start
print(end)
df_error = pd.DataFrame(df_error, columns = ['tweet_id', 'Error'])

1544762173.1630697


Rate limit reached. Sleeping for: 711
Rate limit reached. Sleeping for: 686


1932.1329419612885


In [64]:
# Loop through tweet_json file and extract the tweeet_id, retweet count, likes
df_tweet_likes = []
tweet_file = open('tweet_json.txt')
for line in tweet_file:
    data = json.loads(line)
    tweet_id = data['id']
    retweet_count = data['retweet_count']
    favorite_count = data['favorite_count']
    df_tweet_likes.append({'tweet_id' : tweet_id
                          ,'favorite_count' : favorite_count
                          ,'retweet_count' : retweet_count})

df_tweet_likes = pd.DataFrame(df_tweet_likes, columns = ['tweet_id', 'favorite_count', 'retweet_count'])

In [52]:
len(df_tweet_likes)

2342

## Assess Data

#### Quality
Review each table and find the quality issues
i.e. Data formats, typos, data inconsistencies (abbr vs. full length), missing information, missing records

#### Tidiness
Breaking apart data in columns (email/phone), dog classification(pupper, doggo, ect), table structure

In [8]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
df_tweet_likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
tweet_id          2342 non-null int64
favorite_count    2342 non-null int64
retweet_count     2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


In [10]:
df_error.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
tweet_id    14 non-null int64
Error       14 non-null object
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


In [11]:
# Review the table and see what needs to be cleaned up
df_ta.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [73]:
# Review and locate the number of incorrect names
sorted(df_ta.name.unique())

['Abby',
 'Ace',
 'Acro',
 'Adele',
 'Aiden',
 'Aja',
 'Akumi',
 'Al',
 'Albert',
 'Albus',
 'Aldrick',
 'Alejandro',
 'Alexander',
 'Alexanderson',
 'Alf',
 'Alfie',
 'Alfy',
 'Alice',
 'Amber',
 'Ambrose',
 'Amy',
 'Amélie',
 'Anakin',
 'Andru',
 'Andy',
 'Angel',
 'Anna',
 'Anthony',
 'Antony',
 'Apollo',
 'Aqua',
 'Archie',
 'Arlen',
 'Arlo',
 'Arnie',
 'Arnold',
 'Arya',
 'Ash',
 'Asher',
 'Ashleigh',
 'Aspen',
 'Astrid',
 'Atlas',
 'Atticus',
 'Aubie',
 'Augie',
 'Autumn',
 'Ava',
 'Axel',
 'Bailey',
 'Baloo',
 'Balto',
 'Banditt',
 'Banjo',
 'Barclay',
 'Barney',
 'Baron',
 'Barry',
 'Batdog',
 'Bauer',
 'Baxter',
 'Bayley',
 'BeBe',
 'Bear',
 'Beau',
 'Beckham',
 'Beebop',
 'Beemo',
 'Bell',
 'Bella',
 'Belle',
 'Ben',
 'Benedict',
 'Benji',
 'Benny',
 'Bentley',
 'Berb',
 'Berkeley',
 'Bernie',
 'Bert',
 'Bertson',
 'Betty',
 'Beya',
 'Biden',
 'Bilbo',
 'Billl',
 'Billy',
 'Binky',
 'Birf',
 'Bisquick',
 'Blakely',
 'Blanket',
 'Blipson',
 'Blitz',
 'Bloo',
 'Bloop',
 'Blu',


In [44]:
df_ta.query('tweet_id == 886053434075471873')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [14]:
# Count the number of retweets
len(df_ta.retweeted_status_id.unique())

182

In [15]:
df_ta.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

In [16]:
df_ta.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [17]:
len(df_ta.in_reply_to_status_id.unique())

78

In [18]:
len(df_ta.in_reply_to_user_id.unique())

32

In [62]:
df_tweet_likes['ratio'] = df_tweet_likes.favorite_count / df_tweet_likes.retweet_count

In [63]:
df_tweet_likes.query('ratio < 1')

,tweet_id,favorite_count,retweet_count,source,ratio
31,886054160059072513,0,105,"<a href=""http://twitter.com/download/iphone"" r...",0.0
35,885311592912609280,0,18241,"<a href=""http://twitter.com/download/iphone"" r...",0.0
67,879130579576475649,0,6722,"<a href=""http://twitter.com/download/iphone"" r...",0.0
72,878404777348136964,0,1270,"<a href=""http://twitter.com/download/iphone"" r...",0.0
73,878316110768087041,0,6560,"<a href=""http://twitter.com/download/iphone"" r...",0.0
77,877611172832227328,0,80,"<a href=""http://twitter.com/download/iphone"" r...",0.0
90,874434818259525634,0,14577,"<a href=""http://twitter.com/download/iphone"" r...",0.0
95,873337748698140672,0,1572,"<a href=""http://twitter.com/download/iphone"" r...",0.0
106,871166179821445120,0,5679,"<a href=""http://twitter.com/download/iphone"" r...",0.0
120,868639477480148993,0,2103,"<a href=""http://twitter.com/download/iphone"" r...",0.0


In [54]:
from pprint import pprint
pprint(api.get_status(886054160059072513)._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jul 15 02:45:48 +0000 2017',
 'entities': {'hashtags': [{'indices': [21, 26], 'text': 'BATP'}],
              'symbols': [],
              'urls': [{'display_url': 'twitter.com/dog_rates/stat…',
                        'expanded_url': 'https://twitter.com/dog_rates/status/886053434075471873',
                        'indices': [27, 50],
                        'url': 'https://t.co/WxwJmvjfxo'}],
              'user_mentions': [{'id': 19607400,
                                 'id_str': '19607400',
                                 'indices': [3, 13],
                                 'name': 'Oakland Athletics 🌳🐘⚾️',
                                 'screen_name': 'Athletics'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 886054160059072513,
 'id_str': '886054160059072513',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,

#### Quality Issues
* timestamp not listed as datetime field `df_ta`
* Multiple names not entered correctly (quite, such, a, not, O, just, an, a, all, actually, an, by, getting, his, incredibly, ect) `df_ta`
* remove retweets (182) `df_ta`
* remove tweets that had errors pulling data from twitter api `df_errors`, most likely due to a tweet or account being removed. (14)
* `df_tweet_likes` (166) without a favorite, yet when you visit the page you see they've been favorited.
* Source contains to much non-sense infomation, should be trimmed down to where the tweet came from. (iPhone, twitter, ect) `df_ta`
* Why so many 0 for favorite_count? Looking at data seems maybe from retweets? `df_tweet_likes`

#### Tidy Issues

* Multiple tables `df_ta`, `df_errors`, `df_tweet_likes` and `df_pred`
* Multiple columns for dog rating (doggo, floofer, pupper, puppo) `df_ta`